
A super intelligent mouse is placed in a three dimensional "maze", consisting of a $k\times k\times k$ grid, where the mouse is placed at the beginning at $(1,1,1)$ at time $t=1$ .

Every cell in the maze contains one unit of cheese. The catch: This is temporal cheese; it's phasing in and out, and is present only at certain times. If the mouse is at cell $(a,b,c)$ at time $t$ where the cheese is present in the cell, the mouse collects it. Afterwards, the cheese no longer appears in that cell!

At each time unit, the mouse can either stay in place, or move one step in exactly one direction. The maze has walls allowing only movement right, up, and forward on the X, Y, and Z axes, respectively. For example, if present at $(3,5,1)$ at $t=13$ , the possible locations for the mouse at $t=14$ are

$(3,5,1)$ (**waiting** in place)
$(4,5,1)$ (moving **right** in the X axis)
$(3,6,1)$ (moving **up** in the Y axis)
$(3,5,2)$ (moving **forward** in the Z axis)
We denote the path taken by the mouse by a sequence of letters: 'W', 'R', 'U', and 'F', corresponding to options 1-4 above.

At time $n$ , the mouse is removed from the maze and can no longer collect cheese. His goal: To maximize the cheese collected by then.

The phases of the cheese are determined by the following linear congruential generator:

$f(x) = (ax+c)\ mod\ m$

Where

$a = 1103515245$

$c = 12345$

$m = 2^{31}$

The cheese is present at $(a,b,c)$ at time $t$ if and only if there is a value $0\le x < n/2$ such that $t=f(a\cdot b\cdot c + x)\ mod\ n$ .

For example, when $n=20$ and $(a,b,c) = (3,5,1)$ , the set of values of t where the cheese appears is $\{1, 3, 4, 5, 6, 7, 8, 9, 10, 12\}$ .

For $k=5$ and $n=20$ , the maximum amount of cheese obtainable is 10 units, given by the following path:
FFRFWFWRRRUUUWWWUWW

Your goal: Find the maximum amount of cheese obtainable for $k=30$ and $n=100$ . Supply your solution as two lines, one with the maximum number of cheese units and the other with the path itself.

A bonus "*" will be given for solving the same problem for $k=50$ and $n=200$ , but this time allowing the mouse to move in more possible ways: left (L), down (D) and backwards (B) as well as wait (W), right (R), up (U) and forward (F), and allowing the mouse to collect the cheese more than once for the same cell, at each time where the cheese is present there.

In [ ]:
# Computes whether there is a cheese at (a,b,c,t)
m = 2**31

n = 20
k = 5
def existsCheese(a,b,c,t):
  for x in range(n//2):
    #print(((1103515245*(a*b*c+x)+12345)%m-t)%n)
    if (((1103515245*(a*b*c+x)+12345)%m-t)%n == 0): return True
  return False
class state:
  # a, b, c, t are coordinates of the state, m is number of misses, path tells what previous terms are
  def __init__(self,a,b,c,t,m,path):
    self.a = a
    self.b = b
    self.c = c
    self.t = t
    self.m = m
    self.path = path
    self.code = self.encode()
    self.code2 = self.encode2()
  def expand(self):
    li = []
    if self.a < k: li.append(state(self.a+1,self.b,self.c,self.t+1,self.m+1-1*existsCheese(self.a+1,self.b,self.c,self.t+1),self.path+"R"))
    if self.b < k: li.append(state(self.a,self.b+1,self.c,self.t+1,self.m+1-1*existsCheese(self.a,self.b+1,self.c,self.t+1),self.path+"U"))
    if self.c < k: li.append(state(self.a,self.b,self.c+1,self.t+1,self.m+1-1*existsCheese(self.a,self.b,self.c+1,self.t+1),self.path+"F"))
    li.append(state(self.a,self.b,self.c,self.t+1,self.m+1,self.path+"W"))
    return li
  def goal(self):
    return self.t==n
  # generates a unique code for each state
  # codes with larger value of m will have larger value.
  def encode(self):
    return 2**self.t*3**self.a*5**self.b*7**self.c+7**n*3**n*self.m
  # code 2, stores the coordinates only, used for checking if we visited a state or not already
  def encode2(self):
    return 2**self.t*3**self.a*5**self.b*7**self.c
def checkPath(path):
  claimed=False
  count = 0
  pos = [1,1,1,1]
  for i in path:
    if (claimed==False) and existsCheese(pos[0],pos[1],pos[2],pos[3]):
      print("C",end='')
      count += 1
      claimed=True
    else: print("_",end='')
    pos[3] += 1
    if i == "F": pos[2] += 1
    if i == "U": pos[1] += 1
    if i == "R": pos[0] += 1
    if i != "W": claimed=False
  if existsCheese(pos[0],pos[1],pos[2],pos[3]) and claimed==False:
    print("C")
    count += 1
  else: print("_")
  print("Count = "+str(count))
# for testing.
for t in range(20):
  if (existsCheese(3,5,1,t)): print(str(t)+" ",end = '')
print()
checkPath("FFRFWFWRRRUUUWWWUWW")

1 3 4 5 6 7 8 9 10 12 
___C_C_CCCCCC___CC__
Count = 10


In [ ]:
import heapq

def search_state():
  start = state(1,1,1,1,1-existsCheese(1,1,1,1),"")
  stateDict = dict()
  stateDict[start.code] = start
  MinPQ = []
  heapq.heappush(MinPQ,start.code)
  visited = {start.code2}
  while(True):
    frontCode = heapq.heappop(MinPQ)
    frontState = stateDict[frontCode]
    stateDict.pop(frontCode)
    if frontState.goal():
      print("Goal Found")
      print(frontState.path)
      print(frontState.m)
      print(frontState.t)
      print(len(frontState.path))
      return frontState.path
    for item in frontState.expand():
      #stateDict[item.code] = item
      if not (item.code2 in visited):
        stateDict[item.code] = item
        heapq.heappush(MinPQ,item.code)
        visited.add(item.code2)

n = 100
k = 30
checkPath(search_state())

Goal Found
WWRWWRRWRRWRWWWRRWUUUFUWRRRUUUUFFRFFFURRRRUUURFRRRFRRRUUFFFFRFFUUUFFRUUUFFFUFUURFFUFURRURUUFFFUFFWW
13
100
99
C__C__CC_CC_C___CC_CCCCC_CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC__
Count = 87


In [ ]:
checkPath("WWWWWFFWFFWFWWFFFWUUURUWFUUUUUURRURRRUUUUFFURURRFFUFFFRRFURRFRUUURRFRUUFFRRRFUFRRFURFUUFUFFRRRWRRRW")

C_____CC_CC_C__CCC_CCCCC_CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC_CCC_
Count = 87


In [ ]:
# set to save the values of t for which (a,b,c) has cheese.
n = 200
k = 50
cheeseSet = set()
for a in range(k):
  for b in range(k):
    for c in range(k):
        for x in range(n//2):
          cheeseSet.add((a,b,c,((1103515245*(a*b*c+x)+12345)%m)%n))
def existsCheese2(a,b,c,t):
    if (a,b,c,t) in cheeseSet: return True
    return False
class state2:
  # a, b, c, t are coordinates of the state, m is number of misses, path tells what previous terms are
  def __init__(self,a,b,c,t,m,path):
    self.a = a
    self.b = b
    self.c = c
    self.t = t
    self.m = m
    self.path = path
    self.code = self.encode()
    self.code2 = self.encode2()
  def expand(self):
    li = []
    if self.a < k: li.append(state2(self.a+1,self.b,self.c,self.t+1,self.m+1-existsCheese2(self.a+1,self.b,self.c,self.t+1),self.path+"R"))
    if self.b < k: li.append(state2(self.a,self.b+1,self.c,self.t+1,self.m+1-existsCheese2(self.a,self.b+1,self.c,self.t+1),self.path+"U"))
    if self.c < k: li.append(state2(self.a,self.b,self.c+1,self.t+1,self.m+1-existsCheese2(self.a,self.b,self.c+1,self.t+1),self.path+"F"))
    # new movements
    if self.a > 1: li.append(state2(self.a-1,self.b,self.c,self.t+1,self.m+1-existsCheese2(self.a-1,self.b,self.c,self.t+1),self.path+"L"))
    if self.b > 1: li.append(state2(self.a,self.b-1,self.c,self.t+1,self.m+1-existsCheese2(self.a,self.b-1,self.c,self.t+1),self.path+"D"))
    if self.c > 1: li.append(state2(self.a,self.b,self.c-1,self.t+1,self.m+1-existsCheese2(self.a,self.b,self.c-1,self.t+1),self.path+"B"))
    li.append(state2(self.a,self.b,self.c,self.t+1,self.m+1-existsCheese2(self.a,self.b,self.c,self.t+1),self.path+"W"))
    return li
  def goal(self):
    return self.t==n
  def encode(self):
    return 2**self.t*3**self.a*5**self.b*7**self.c+7**n*3**n*self.m
  # code 2, stores the coordinates only, used for checking if we visited a state or not already
  def encode2(self):
    return 2**self.t*3**self.a*5**self.b*7**self.c
def checkPath2(path, cheeseSet):
  count = 0
  pos = [1,1,1,1]
  for i in path:
    if (pos[0],pos[1],pos[2],pos[3]) in cheeseSet:
      print("C")
      count += 1
    else: print("_",end='')
    pos[3] += 1
    if i == "F": pos[2] += 1
    if i == "U": pos[1] += 1
    if i == "R": pos[0] += 1
    if i == "L": pos[2] -= 1
    if i == "D": pos[1] -= 1
    if i == "B": pos[0] -= 1
  if (pos[0],pos[1],pos[2],pos[3]) in cheeseSet:
    print("C")
    count += 1
  else: print("_")
  print("Count = "+str(count))

In [ ]:
start = state2(1,1,1,1,1,"")
#print(start)
# iterative deepening
max_loss = 0
found = False
while(found == False):
  visited = {start.code2:start.m}
  objects_expanded = 0
  li = [start]
  max_depth = 0
  while (len(li)>0):
    front = li.pop()
    max_depth = max(max_depth, front.t)
    if front.m > max_loss:
      print(max_loss)
      print(front.path)
    if front.goal():
        print(front.path)
        print(n-front.m)
        #if not found: max_loss -= 1
        found = True
        break
    if front.m < max_loss:
      for state in front.expand():
        if state.code2 not in visited.keys() or state.m < visited[state.code2]:
          li.append(state)
          visited[state.code2] = state.m
      objects_expanded += 1
  print("Max Loss "+str(max_loss)+", Max Depth "+str(max_depth)+", Objects expanded: "+str(objects_expanded))
  max_loss+=1
print("Max Loss "+str(max_loss)+", Objects expanded: "+str(objects_expanded))

0

Max Loss 0, Max Depth 1, Objects expanded: 0
Max Loss 1, Max Depth 1, Objects expanded: 0
Max Loss 2, Max Depth 3, Objects expanded: 5
Max Loss 3, Max Depth 4, Objects expanded: 15
Max Loss 4, Max Depth 6, Objects expanded: 70
Max Loss 5, Max Depth 8, Objects expanded: 210
Max Loss 6, Max Depth 9, Objects expanded: 330
Max Loss 7, Max Depth 11, Objects expanded: 617
Max Loss 8, Max Depth 13, Objects expanded: 1213
Max Loss 9, Max Depth 16, Objects expanded: 2599
Max Loss 10, Max Depth 18, Objects expanded: 4999
Max Loss 11, Max Depth 21, Objects expanded: 8956
Max Loss 12, Max Depth 23, Objects expanded: 14653
Max Loss 13, Max Depth 29, Objects expanded: 23292
Max Loss 14, Max Depth 40, Objects expanded: 37857
Max Loss 15, Max Depth 49, Objects expanded: 68607
WFFFFFFFFFFFUFFFFUURFFUUUUUFRLLUURFRWRDRWRBRLLRWRWUWRDLUWRLLRLLRLLRWRLLRLLRWRBRWRBRWRWUBLUWUWRWFWRWFWFWRBRWUWFBLLFBDFWFWUWFWLFWFWLUWUBLLLLLLLRWRLLRBRLLRDRLLRDLRLRDRDRLLRBRLLRBRBRBRBRWUWRWRBRWRWFWRDLW
184
Max Loss 16, Max Depth

In [ ]:
def checkPath2(path, cheeseSet):
  count = 0
  pos = [1,1,1,1]
  for i in range(len(path)):
    if (pos[0],pos[1],pos[2],pos[3]) in cheeseSet:
      print("C",end='')
      count += 1
    else: print("_",end='')
    pos[3] += 1
    if path[i] == "F": pos[2] += 1
    if path[i] == "U": pos[1] += 1
    if path[i] == "R": pos[0] += 1
    if path[i] == "B": pos[2] -= 1
    if path[i] == "D": pos[1] -= 1
    if path[i] == "L": pos[0] -= 1
  if (pos[0],pos[1],pos[2],pos[3]) in cheeseSet:
    print("C")
    count += 1
  else: print("_")
  print(pos[3])
  print("Count = "+str(count))
path2 = "WFFFFFFFFFFFUFFFFUURFFUUUUUFRLLUURFRWRDRWRBRLLRWRWUWRDLUWRLLRLLRLLRWRLLRLLRWRBRWRBRWRWUBLUWUWRWFWRWFWFWRBRWUWFBLLFBDFWFWUWFWLFWFWLUWUBLLLLLLLRWRLLRBRLLRDRLLRDLRLRDRDRLLRBRLLRBRBRBRBRWUWRWRBRWRWFWRDLW"
checkPath2(path2,cheeseSet)

_C__C_C__C_C_CC_C_C_CC_C_C_CCCCC_CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC_
200
Count = 184
